In [2]:
import os
import time
import urllib
import multiprocessing
import pandas as pd
from pathlib import Path
from PIL import Image
from functools import partial
from tqdm import tqdm

In [3]:
import os
import re
import json
import wandb
import torch
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from tqdm import tqdm
from torch.utils.data import Dataset
from transformers import TrainingArguments, TrainerCallback
from unsloth import FastVisionModel
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer

/tmp/ipykernel_221849/4029548990.py:13: UserWarning: WARNING: Unsloth should be imported before [transformers, peft] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastVisionModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
!pip install unsloth wandb pillow pandas tqdm
!pip install git+https://github.com/huggingface/transformers  # latest for Qwen2-VL

INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 141.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 256.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 21.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 121.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 318.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 320.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 23.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 351.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.7/803.7 kB 46.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 85.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 130.3 MB/s  0:00:00
   ━━━━━━━━

In [4]:
import zipfile
import os

In [5]:
zip_path = "my_images.zip"
extract_to = "images"

In [7]:
if not os.path.exists(extract_to):
    os.makedirs(extract_to)

print("Extracting images...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
print(f"Done! Extracted to {extract_to}")

Extracting images...


Done! Extracted to images


In [6]:
TRAIN_CSV       = "small_train.csv"
VAL_CSV         = "small_val.csv"
IMAGE_DIR       = "images/"
OUTPUT_DIR      = "./qwen2vl_amazon_lora"
MODEL_NAME = "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit"

In [7]:
WANDB_PROJECT   = "amazon-ml-entity-extraction"
WANDB_RUN_NAME  = "qwen2vl-7b-lora-r32"

In [8]:

LORA_R      = 16
LORA_ALPHA   = 32
LORA_DROPOUT    = 0.05

In [9]:
BATCH_SIZE  = 2
GRAD_ACCUM  = 8
MAX_SEQ_LEN  = 256     # down from 512
LEARNING_RATE   = 2e-4
NUM_EPOCHS      = 3
MAX_SEQ_LEN     = 256
WARMUP_RATIO    = 0.05
EVAL_STEPS      = 500
SAVE_STEPS      = 500
LOG_STEPS       = 50
# ─────────────────────────────────────────────────────────────────────────────

In [10]:
# ── ALLOWED UNITS (from competition) ─────────────────────────────────────────
ALLOWED_UNITS = {
    "centimetre", "foot", "inch", "metre", "millimetre", "yard",
    "cubic foot", "cubic inch", "cup", "decilitre", "fluid ounce",
    "gallon", "imperial gallon", "litre", "microlitre", "millilitre",
    "pint", "quart",
    "gram", "kilogram", "microgram", "milligram", "ounce", "pound", "ton",
    "kilovolt", "millivolt", "volt",
    "kilowatt", "watt",
    "decibel",
    "centilitre",
    "cubic centimetre",
    "tonne",
}

In [11]:
# ── F1 METRIC ─────────────────────────────────────────────────────────────────
def compute_f1(predictions: list[str], ground_truths: list[str]) -> dict:
    """
    Compute F1 score using competition logic.
    TP: pred != "" and gt != "" and pred == gt
    FP: (pred != "" and gt != "" and pred != gt) OR (pred != "" and gt == "")
    FN: pred == "" and gt != ""
    TN: pred == "" and gt == ""
    """
    tp, fp, fn = 0, 0, 0
    for pred, gt in zip(predictions, ground_truths):
        pred = pred.strip()
        gt   = gt.strip()
        if pred != "" and gt != "" and pred == gt:
            tp += 1
        elif pred != "" and gt != "" and pred != gt:
            fp += 1
        elif pred != "" and gt == "":
            fp += 1
        elif pred == "" and gt != "":
            fn += 1
        # TN: both empty — no effect on F1

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1        = (2 * precision * recall / (precision + recall)
                 if (precision + recall) > 0 else 0.0)
    return {"f1": f1, "precision": precision, "recall": recall, "tp": tp, "fp": fp, "fn": fn}

In [12]:
# ── PROMPT BUILDER ────────────────────────────────────────────────────────────
def build_prompt(entity_name: str) -> str:
    units_str = ", ".join(sorted(ALLOWED_UNITS))
    return (
        f"You are an AI assistant that extracts product attributes from e-commerce images.\n\n"
        f"Task: Extract the value of '{entity_name}' from this product image.\n\n"
        f"Rules:\n"
        f"1. Return ONLY the numeric value followed by a single space and the unit.\n"
        f"2. The unit MUST be one of these allowed units: {units_str}\n"
        f"3. If the entity cannot be found in the image, return an empty string.\n"
        f"4. Do NOT include any explanation, punctuation, or extra text.\n\n"
        f"Examples of valid output: '500.0 gram', '1.0 cup', '27.5 centimetre'\n"
        f"Examples of invalid output: '500g', '500 grams', 'approximately 500 gram'\n\n"
        f"Entity to extract: {entity_name}\n"
        f"Answer:"
    )


In [13]:
def make_conversation(image_path: str, entity_name: str, entity_value: str = None):
    prompt = build_prompt(entity_name)

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},   # ← key must be "image"
                {"type": "text",  "text": prompt},
            ],
        }
    ]

    if entity_value is not None:
        messages.append({
            "role": "assistant",
            "content": [{"type": "text", "text": str(entity_value)}],
        })

    return messages

In [14]:
# ── DATASET ───────────────────────────────────────────────────────────────────
class AmazonMLDataset(Dataset):
    def __init__(self, csv_path: str, image_dir: str, split: str = "train"):
        self.df        = pd.read_csv(csv_path)
        self.image_dir = image_dir
        self.split     = split
        print(f"[{split}] Loaded {len(self.df)} samples")

    def __len__(self):
        return len(self.df)

    def _get_image_path(self, image_link: str) -> str:
        filename = Path(image_link).name
        return os.path.join(self.image_dir, filename)
    

    def __getitem__(self, idx):
        row          = self.df.iloc[idx]
        image_link   = row["image_link"]
        filename     = Path(image_link).name
        image_path   = f"/teamspace/studios/this_studio/images/{filename}"
        entity_name  = row["entity_name"]
        entity_value = str(row["entity_value"]) if self.split != "test" else None
    
        # Fallback to placeholder if image missing
        if not os.path.exists(image_path):
            image_path = "/teamspace/studios/this_studio/images/placeholder.jpg"
            # create a placeholder if it doesn't exist
            if not os.path.exists(image_path):
                from PIL import Image as PILImage
                PILImage.new('RGB', (100, 100), color='black').save(image_path)
    
        return {"messages": make_conversation(image_path, entity_name, entity_value)}

In [15]:
# ── W&B F1 CALLBACK ───────────────────────────────────────────────────────────
class WandbF1Callback(TrainerCallback):
    """
    Runs evaluation on val set every eval_steps and logs F1 to W&B.
    Unsloth's SFTTrainer doesn't natively compute custom metrics so we do it manually.
    """

    def __init__(self, model, processor, val_df, image_dir, eval_samples=500):
        self.model       = model
        self.processor   = processor
        self.val_df      = val_df.sample(min(eval_samples, len(val_df)), random_state=42)
        self.image_dir   = image_dir
        self.best_f1     = 0.0

    def _predict_single(self, image_path, entity_name):
        prompt = build_prompt(entity_name)
    
        if not os.path.exists(image_path):
            return ""   # ← just return empty string, skip missing images
    
        try:
            image = Image.open(image_path).convert("RGB")
        except Exception:
            return ""
    
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": prompt},
                ],
            }
        ]
    
        text_prompt = self.processor.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        inputs = self.processor(
            text=[text_prompt],
            images=[image],
            return_tensors="pt",
            padding=True,
        ).to(self.model.device)
    
        with torch.no_grad():
            output_ids = self.model.generate(
                **inputs,
                max_new_tokens=32,
                do_sample=False,
                temperature=1.0,
                use_cache=True,
            )
    
        generated = output_ids[0][inputs["input_ids"].shape[1]:]
        return self.processor.decode(generated, skip_special_tokens=True).strip()


    def on_evaluate(self, args, state, control, **kwargs):
        print(f"\n[W&B F1 Callback] Running F1 eval on {len(self.val_df)} samples...")
        self.model.eval()
    
        predictions   = []
        ground_truths = []
    
        for _, row in tqdm(self.val_df.iterrows(), total=len(self.val_df), desc="F1 Eval"):
            image_path = f"/teamspace/studios/this_studio/images/{Path(row['image_link']).name}"
            pred       = self._predict_single(image_path, row["entity_name"])
            predictions.append(pred)
            ground_truths.append(str(row["entity_value"]).strip())
    

In [16]:
import wandb
wandb.login(key="jdoebo",relogin=True)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: Appending key for api.wandb.ai to your netrc file: /teamspace/studios/this_studio/.netrc
wandb: Currently logged in as: sudhanvaiitr (sudhanvaiitr-indian-institute-of-technology-roorkee) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [17]:
from trl import SFTTrainer, SFTConfig

In [19]:
# ── MAIN TRAINING SCRIPT ──────────────────────────────────────────────────────
def main():
    # 1. Init W&B
    wandb.init(
        project=WANDB_PROJECT,
        name=WANDB_RUN_NAME,
        config={
            "model":         MODEL_NAME,
            "lora_r":        LORA_R,
            "lora_alpha":    LORA_ALPHA,
            "batch_size":    BATCH_SIZE,
            "grad_accum":    GRAD_ACCUM,
            "learning_rate": LEARNING_RATE,
            "epochs":        NUM_EPOCHS,
            "max_seq_len":   MAX_SEQ_LEN,
        },
    )

    # 2. Load model + processor via Unsloth
    print("Loading model...")
    model, processor = FastVisionModel.from_pretrained(
        MODEL_NAME,
        load_in_4bit=True,       # QLoRA — essential for 
        use_gradient_checkpointing="unsloth",
    )

    # 3. Apply LoRA via Unsloth
    model = FastVisionModel.get_peft_model(
        model,
        finetune_vision_layers     = True,   # fine-tune vision encoder too
        finetune_language_layers   = True,   # fine-tune LLM backbone
        finetune_attention_modules = True,
        finetune_mlp_modules       = True,
        r           = LORA_R,
        lora_alpha  = LORA_ALPHA,
        lora_dropout= LORA_DROPOUT,
        bias        = "none",
        use_dora    = False,                 # set True to try DoRA
        random_state= 42,
    )

    # 4. Datasets
    train_dataset = AmazonMLDataset(TRAIN_CSV, IMAGE_DIR, split="train")
    val_df        = pd.read_csv(VAL_CSV)

    # 5. Training args
    training_args = TrainingArguments(
        output_dir                  = OUTPUT_DIR,
        num_train_epochs            = NUM_EPOCHS,
        per_device_train_batch_size = BATCH_SIZE,
        gradient_accumulation_steps = GRAD_ACCUM,
        learning_rate               = LEARNING_RATE,
        warmup_ratio                = WARMUP_RATIO,
        lr_scheduler_type           = "cosine",
        fp16                        = not torch.cuda.is_bf16_supported(),
        bf16                        = torch.cuda.is_bf16_supported(),
        logging_steps               = LOG_STEPS,
        eval_strategy               = "steps",
        eval_steps                  = EVAL_STEPS,
        save_strategy               = "steps",
        save_steps                  = SAVE_STEPS,
        save_total_limit            = 3,
        load_best_model_at_end      = False,   # custom F1 metric, not loss
        report_to                   = "wandb",
        dataloader_num_workers    = 4,      # up from 2
        dataloader_pin_memory     = True,   # speeds up CPU→GPU transfer
        dataloader_prefetch_factor = 2,     # prefetch batches in advance
        remove_unused_columns       = False,
        optim                       = "adamw_8bit",  # memory efficient
        weight_decay                = 0.01,
        max_grad_norm               = 1.0,
        seed                        = 42,
    )

    # 6. W&B F1 callback
    f1_callback = WandbF1Callback(
        model      = model,
        processor  = processor,
        val_df     = val_df,
        image_dir  = IMAGE_DIR,
        eval_samples = 50,   # use 300 samples per eval for speed on 
    )

    # 7. Trainer
    def formatting_func(examples):
        texts = []
        for msg in examples["messages"]:
            text_str = processor.apply_chat_template(
                msg,
                tokenize=False,
                add_generation_prompt=False,
            ).strip()
            texts.append(text_str)
        return {"text": texts}
    
    trainer = SFTTrainer(
        model           = model,
        processing_class   = processor,
        data_collator   = UnslothVisionDataCollator(model, processor),
        train_dataset   = train_dataset,
        args = SFTConfig (
            output_dir                  = OUTPUT_DIR,
            num_train_epochs            = NUM_EPOCHS,
            per_device_train_batch_size = BATCH_SIZE,
            gradient_accumulation_steps = GRAD_ACCUM,
            learning_rate               = LEARNING_RATE,
            warmup_ratio                = WARMUP_RATIO,
            lr_scheduler_type           = "cosine",
            fp16                        = not torch.cuda.is_bf16_supported(),
            bf16                        = torch.cuda.is_bf16_supported(),
            logging_steps               = LOG_STEPS,
            save_strategy               = "steps",
            save_steps                  = SAVE_STEPS,
            save_total_limit            = 3,
            report_to                   = "wandb",
            optim                       = "adamw_8bit",
            weight_decay                = 0.01,
            max_grad_norm               = 1.0,
            seed                        = 42,
            max_seq_length              = MAX_SEQ_LEN,
            packing                     = False,
            dataloader_num_workers      = 4,      # ← now actually being used
            dataloader_pin_memory       = True,
            dataloader_prefetch_factor  = 2,
        ), # it is showing "No paramter named tokeinzer at this point in red underlined"
        callbacks       = [f1_callback],
    )
    # 8. Train!
    print("Starting training...")
    trainer.train()

    # 9. Save final model
    model.save_pretrained(OUTPUT_DIR)
    processor.save_pretrained(OUTPUT_DIR)
    print(f"Model saved to {OUTPUT_DIR}")

    wandb.finish()


if __name__ == "__main__":
    main()

Loading model...
==((====))==  Unsloth 2026.2.1: Fast Qwen2_Vl patching. Transformers: 5.3.0.dev0.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.278 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.35. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading weights:   0%|          | 0/729 [00:00<?, ?it/s]

Unsloth: Making `model.base_model.model.model.visual` require gradients


warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.
warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


[train] Loaded 9996 samples
Unsloth: Model does not have a default image size - using 512


...

Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,996 | Num Epochs = 3 | Total steps = 1,875
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 28,950,528 of 2,237,936,128 (1.29% trained)


Step,Training Loss
50,1.081628
100,0.020192
150,0.012509
200,0.011763
250,0.011296
300,0.012041
350,0.010663
400,0.010736
450,0.010374
500,0.009701


train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,▂▂▁▂▂▁▂▂▁▁▁▁▃▁▁▂▁▂▁▁█▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▅███████▇▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
train/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_flos,2.257099452293284e+17
train/epoch,3
train/global_step,1875
train/grad_norm,0.02603
train/learning_rate,0.0
train/loss,0.0069


Model saved to ./qwen2vl_amazon_lora
